In [13]:
import yfinance as yf
import pandas as pd
import requests
import json
import re

from selectolax.parser import HTMLParser
from bs4 import BeautifulSoup

In [3]:
def parse_header(raw_header: str):
    header = dict()

    for line in raw_header.split("\n"):

        if line.startswith(":"):
            a, b = line[1:].split(":", 1)
            a = f":{a}"
        else:
            a, b = line.split(":",1)

        header[a.strip()] = b.strip()

    return header

In [4]:
header_str = """accept: */*
accept-encoding: gzip, deflate, br
accept-language: ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7
cookie: A1=d=AQABBObjlWMCELURSo76ZewBp_V0DINbhUEFEgABCAGNOmRiZOS2b2UB9qMAAAcI5uOVY4NbhUE&S=AQAAAhCn1sRdFt9rlJfAvARe0a4; A3=d=AQABBObjlWMCELURSo76ZewBp_V0DINbhUEFEgABCAGNOmRiZOS2b2UB9qMAAAcI5uOVY4NbhUE&S=AQAAAhCn1sRdFt9rlJfAvARe0a4; B=431argdhpbov6&b=3&s=m6; GUC=AQABCAFkOo1kYkIdhwSM; A1S=d=AQABBObjlWMCELURSo76ZewBp_V0DINbhUEFEgABCAGNOmRiZOS2b2UB9qMAAAcI5uOVY4NbhUE&S=AQAAAhCn1sRdFt9rlJfAvARe0a4&j=GDPR; cmp=t=1682767671&j=1&u=1---&v=79; EuConsent=CPmFzsAPmFzsAAOACBESDBCoAP_AAEfAACiQJMtd_H__bX9v-f7_6ft0eY1f9_rz7uQzDhfNk-4F3L_W_LwX_2E7NF36tq4KmR4ku1DBIUNtHMnUDUmxaokVrzHsak2cpTNKJ-BkkHMZe2dYGF5vm5tj-QKZ5v5_d3f52T_9_dv-39z33913v3d93-_12PjdU5_9H_v_fR_b8_Kf9_7-_4v8_9_rk2_e3_____9v__4JMgEmGrcQBdiWOBNoGEUCIEYVhIRQKACCgGFogIAHBwU7KwCfWECABAKAIwIgQ4AowABAAABAEhEAEgRYIAAARAIAAQAIBEIACBgEFABYGAQAAgGgYohQACBIAZEBEUpgQEQJBAS2VCCUF0hphAFWWAFAIjYKABEAAArAAEBYOAYIkBKxYIEmINogAGCFAKJUK1BJ6YAYaADAAEEpxEAGAAIJTioAMAAQSnA; PRF=t%3D000069.SZ%252B000155.SZ%252B600369.SS%252BEME%252B2333.HK%252BBIDU%252BBEKE%26newChartbetateaser%3D1
origin: https://finance.yahoo.com
referer: https://finance.yahoo.com/quote/000069.SZ?p=000069.SZ&.tsrc=fin-srch
sec-ch-ua: "Chromium";v="112", "Google Chrome";v="112", "Not:A-Brand";v="99"
sec-ch-ua-mobile: ?0
sec-ch-ua-platform: "macOS"
sec-fetch-dest: empty
sec-fetch-mode: cors
sec-fetch-site: same-site
user-agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"""

In [5]:
headers = parse_header(header_str)

In [6]:
with open("headers.json", "w") as file:
    json.dump(headers, file)

In [30]:
params = {
    "q": "KEDA CLEAN ENERGY",
    "lang": "en-US",
    "region": "US",
    "quotesCount": 6,
    "enableFuzzyQuery": "false",
    "quotesQueryId": "tss_match_phrase_query",
    "multiQuoteQueryId": "multi_quote_single_token_query",
    "newsQueryId": "news_cie_vespa",
    "enableCb": "true",
    "enableNavLinks": "true",
    "enableEnhancedTrivialQuery": "true",
    "enableResearchReports": "true",
    "enableCulturalAssets": "true",
    "enableLogoUrl": "true",
    "researchReportsCount": 2
}

In [35]:
resp = requests.get(
    "https://query1.finance.yahoo.com/v1/finance/search",
    params=params,
    headers=headers
)

In [36]:
resp.text

'<!doctype html public "-//W3C//DTD HTML 4.01//EN" "http://www.w3.org/TR/html4/strict.dtd">\n<html>\n<head>\n    <title>Yahoo! - Error report</title>\n    <style>\n        /* nn4 hide */ /*/*/\n        body {\n        font: small/ 1.2em arial, helvetica, clean, sans-serif;\n        font: x-small;\n        text-align: center;\n        }\n\n        table {\n        font-size: inherit;\n        font: x-small;\n        }\n\n        html>body {\n        font: 83%/ 1.2em arial, helvetica, clean, sans-serif;\n        }\n\n        input {\n        font-size: 100%;\n        vertical-align: middle;\n        }\n\n        p,form {\n        margin: 0;\n        padding: 0;\n        }\n\n        p {\n        padding-bottom: 6px;\n        margin-bottom: 10px;\n        }\n\n        #doc {\n        width: 48.5em;\n        margin: 0 auto;\n        border: 1px solid #fff;\n        text-align: center;\n        }\n\n        #ygma {\n        text-align: right;\n        margin-bottom: 53px\n        }\n\n     

In [11]:
quote = resp["quotes"][0]["symbol"]

TypeError: 'Response' object is not subscriptable

In [47]:
params = {
    "lang": "en-US",
    "region": "US",
    "symbol": "3968.HK",
    "padTimeSeries": "true",
    "type": ",".join(of_interest),
    "merge": "false",
    "period1": 493590046,
    "period2": 1682771124,
    "corsDomain": "finance.yahoo.com"
}

In [49]:
# https://finance.yahoo.com/quote/3968.HK/cash-flow?p=3968.HK

In [61]:
res = requests.get(
    "https://query1.finance.yahoo.com/ws/fundamentals-timeseries/v1/finance/timeseries/UK?lang=en-US&region=US&symbol=UK&padTimeSeries=true&type=annualTotalRevenue%2CannualBasicEPS%2CannualTotalAssets%2CannualNetIncome%2CannualTotalLiabilitiesNetMinorityInterest%2CannualTotalEquityGrossMinorityInterest%2CannualInvestingCashFlow%2CannualTotalCapitalization&merge=false&period1=493590046&period2=1682771124&corsDomain=finance.yahoo.com",
    headers=headers
).json()

In [50]:
resp = requests.get(
    "https://query1.finance.yahoo.com/ws/fundamentals-timeseries/v1/finance/timeseries/3968.HK",
    headers=headers,
    params=params
).json()

In [52]:
of_interest = [
    "annualTotalRevenue", "annualBasicEPS", "annualTotalAssets", 
    "annualNetIncome", "annualTotalLiabilitiesNetMinorityInterest",
    "annualTotalEquityGrossMinorityInterest", "annualInvestingCashFlow",
    "annualTotalCapitalization"
]

In [62]:
[
    res["meta"]["type"][0] for res in resp["timeseries"]["result"]
]

['annualNetIncome',
 'annualTotalLiabilitiesNetMinorityInterest',
 'annualTotalEquityGrossMinorityInterest',
 'annualTotalRevenue',
 'annualTotalCapitalization',
 'annualBasicEPS',
 'annualTotalAssets',
 'annualInvestingCashFlow']

In [22]:
bool("")

False

In [41]:
from datetime import datetime

In [43]:
datetime.strptime("2021-10-02", "%Y-%M-%d").year

2021

In [6]:
resp = requests.get(
    "https://finance.yahoo.com/quote/605117.SS/profile?p=605117.SS",
    headers=headers
)

In [20]:
soup = BeautifulSoup(resp.text, "html.parser")
soup.find("span", string=re.compile("Full")).findNext().text

'3,791'